# NetR9 Reprojection

In [1]:
from import_df import NmeaDf
import swiss_projection
import numpy as np
import pandas as pd

In [2]:
def degmin2deg(degmin):
    a, b = str(degmin).split('.')
    d = int(a[:-2])
    m = float(a[-2:] + '.'+ b)

    degrees = d + m/60
    #rad = degrees*180/np.pi
    return degrees

In [8]:
# dataframe with the coordinates
filepath = "../2021/January/29/swipos_NetR9.nmea"
timeseries = NmeaDf(filepath)
df = timeseries.getDataFrame()

# Drop rows where there is no coordinates
df = df.dropna(subset=['lat', 'lon', 'lat']).reset_index(drop=True)

## GLL
| name      | Format     | example    |
|-----------|------------|------------|
| latitute  | ddmm.mmmmm | 4717.11364 |
| longitute | ddmm.mmmmm | 09231.00   |
| height    |            |            |

array([[4610.48513929],
       [4610.4851066 ],
       [4610.48509111],
       ...,
       [4610.51781052],
       [4610.52165115],
       [4610.51093071]])

In [9]:
# Dataframe to matrix
lon =  pd.DataFrame(df['lon']).to_numpy()
lat =  pd.DataFrame(df['lat']).to_numpy()
alt =  pd.DataFrame(df['alt']).to_numpy()

# Creation of a matrix len(lat) x 3
#llh = np.concatenate((lon,lat,alt), axis=1)
# Converstion to lv95
lv95 = np.zeros((len(lon),3)) # Pre-allocate matrix
for i in range(len(alt)):
    # Convert in degree (nmea format)
    llh = [degmin2deg(lon[i].tolist()[0]), degmin2deg(lat[i].tolist()[0]), alt[i].tolist()[0]]
    
    lv95[i, :] = swiss_projection.wgs84_to_lv95(llh)

In [6]:
# convert array into dataframe 
df_out = pd.DataFrame(lv95, columns=['Latitude', 'Longitude', 'Altitude']) 
df_out['timestamp']=df['timestamp']
df_out['diffAgeGP']=df['diffAgeGP']
df_out['fixMode']=df['posMode']

# save the dataframe as a csv file 
df_out.to_csv("../tracks/sapcorda.csv")